# Visualizing cone calorimeter data
Note: for best results, view this notebook in VSCode with the Jupyter extension installed, not JupyterLab.

In [24]:
# install dependencies
%pip install pandas
%pip install plotly
%pip install scipy
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [25]:
import pandas as pd
import scipy

import json
from pathlib import Path
from datetime import date

import ipywidgets as widgets
from IPython.display import display, clear_output

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [26]:
def colorize(text, color):
    if color == "red":
        color = 196
    elif color == "green":
        color = 45
    elif color == "blue":
        color = 80
    elif color == "yellow":
        color = 226
    elif color == "purple":
        color = 92
    elif color == "cyan":
        color = 123
    elif color == "white":
        color = 15
    elif color == "black":
        color = 0
    else:
        color = 15

    return f"\x1b[38;5;{color}m{text}\x1b[0m"

In [45]:
# some basic setup
out = widgets.Output()

input_path = Path("../OUTPUT/")

source_widget = widgets.Dropdown(options=[x.name for x in input_path.iterdir() if x.is_dir()], description="Source:")

metadata_files = list(Path(input_path / Path(source_widget.value)).glob("**/*.json"))

filename_widget = widgets.Dropdown(options=[str(x.stem).replace("_metadata", "") for x in metadata_files], description="Filename:")

data_file_path = Path(filename_widget.value + "_data.csv")
data = pd.read_csv(input_path / Path(source_widget.value) / data_file_path)

# UI code

def update_folder(event=None):
    # get test metadata
    global metadata_files
    metadata_files = list(Path(input_path / Path(source_widget.value)).glob("**/*.json"))

    global filename_widget
    filename_widget = widgets.Dropdown(options=[str(x.stem).replace("_metadata", "") for x in metadata_files], description="Filename:")

    print("filename")

    update_data()

# Select X & Y axis
x_axis_widget = widgets.Dropdown(description='X axis:', options=data.columns)
y_axis_widget = widgets.Dropdown(description='Y axis:', options=data.columns)
sec_y_axis_widget = widgets.Dropdown(description='Sec. Y axis:', options=data.columns)
sec_x_axis_widget = widgets.Dropdown(description='Sec. X axis:', options=data.columns)

y_max = widgets.FloatText(description='Y max:', value=0)
y_min = widgets.FloatText(description='Y min:', value=0)
y_scale_enabled = widgets.Checkbox(description='Y scale:', value=False)
savgol_filter_enabled = widgets.Checkbox(description='Savgol filter', value=False)

sec_y_max = widgets.FloatText(description='Sec. Y max:', value=0)
sec_y_min = widgets.FloatText(description='Sec. Y min:', value=0)
sec_y_scale_enabled = widgets.Checkbox(description='Sec. Y scale:', value=False)
sec_savgol_filter_enabled = widgets.Checkbox(description='Savgol filter', value=False)

update = widgets.Button(description="Plot", icon="check")

fig = make_subplots(specs=[[{"secondary_y": True}]])

def update_data(event=None):
    global data, data_file_path

    data_file_path = Path(filename_widget.value + "_data.csv")
    print(data_file_path)
    data = pd.read_csv(input_path / Path(source_widget.value) / data_file_path)

    x_axis_widget.options = data.columns
    y_axis_widget.options = data.columns

    sec_y_axis_widget.options = data.columns
    sec_x_axis_widget.options = data.columns

    plot()

def plot(event=None):

    clear_output()

    primary_box = widgets.Box(children=[x_axis_widget, y_axis_widget, y_scale_enabled, y_min, y_max, savgol_filter_enabled])
    secondary_box = widgets.Box(children=[sec_x_axis_widget, sec_y_axis_widget, sec_y_scale_enabled, sec_y_min, sec_y_max, sec_savgol_filter_enabled])
    vbox = widgets.VBox([source_widget, filename_widget, primary_box, secondary_box, update])

    display(vbox)

    y_data = data[y_axis_widget.value]
    x_data = data[x_axis_widget.value]

    sec_y_data = data[sec_y_axis_widget.value]
    sec_x_data = data[sec_x_axis_widget.value]

    # Optionally add savgol filter to the data
    if savgol_filter_enabled.value:
        y_data = scipy.signal.savgol_filter(y_data, 51, 3)
    if sec_savgol_filter_enabled.value:
        sec_y_data = scipy.signal.savgol_filter(sec_y_data, 51, 3)

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    if y_axis_widget.value != x_axis_widget.value:
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines', name=f"{y_axis_widget.value} vs {x_axis_widget.value}"), secondary_y=False)
        if y_scale_enabled.value:
            fig.update_yaxes(range=[y_min.value, y_max.value], autorange=False)
    
    if sec_y_axis_widget.value != sec_x_axis_widget.value:
        fig.add_trace(go.Scatter(x=sec_x_data, y=sec_y_data, mode='lines', name=f"{sec_y_axis_widget.value} vs {sec_x_axis_widget.value} (secondary)"), secondary_y=True)
        if sec_y_scale_enabled.value:
            fig.update_yaxes(range=[sec_y_min.value, sec_y_max.value], secondary_y=True, autorange=False)

    fig.show()

# TODO: also allow setting the scale for each axis

filename_widget.observe(update_data, names="value")
source_widget.observe(update_folder, names="value")


update.on_click(plot)

plot()


In [33]:
# print out material info again:

metadata_files = list(Path(input_path / Path(source_widget.value)).glob("**/*.json"))

for metadata_file in metadata_files:
    test_name = metadata_file.stem.replace("_metadata", "")
    metadata = json.load(metadata_file.open())
    print(colorize(f"Selected test: {test_name}", "blue"))
    print(f" - Date: {metadata.get('date')}")
    print(f" - Heat flux (kw/m^2): {metadata.get('heat_flux_kw/m^2')}")
    print(f" - Material: {metadata.get('material_name')}")
    print(f" - Specimen description: {metadata.get("specimen_description")}")
    print(f" - General comments: {metadata.get('comments')}")

Selected test: 8-5-2019-CONELAB-PVC-Fen-1-scaled
 - Date: 2019-05-08T14:09:40
 - Heat flux (kw/m^2): None
 - Material: None
 - Specimen description: None
 - General comments: PVC Fence
50 kW/m2
Selected test: 8-5-2019-CONELAB-PVC-Fen-2-scaled
 - Date: 2019-05-08T14:47:25
 - Heat flux (kw/m^2): None
 - Material: None
 - Specimen description: None
 - General comments: PVC Fence
50 kW/m2
Selected test: 8-5-2019-CONELAB-PVC-Fen-3-scaled
 - Date: 2019-05-08T15:10:24
 - Heat flux (kw/m^2): None
 - Material: None
 - Specimen description: None
 - General comments: PVC Fence
50 kW/m2
Selected test: 8-6-2019-CONELAB-Wood-Fen-1-scaled
 - Date: 2019-06-08T13:11:15
 - Heat flux (kw/m^2): None
 - Material: None
 - Specimen description: None
 - General comments: Wood Fence
50 kW/m2
Selected test: 8-6-2019-CONELAB-Wood-Fen-2-scaled
 - Date: 2019-06-08T14:38:15
 - Heat flux (kw/m^2): None
 - Material: None
 - Specimen description: None
 - General comments: Wood Fence
50 kW/m2
Selected test: 8-6-2019-CO